# Intern Roster

## Introduction

Monash Health has given us with the oppourtunity to take on the task of creating an Intern Roster for their new intake of Interns in the pharmacy department.


We have 11 interns. Let each intern be $i$.

We have 13 rotations.  Let each rotation be $j$.  There are also three annual leave rotations.  These shall be $j$ values 14, 15, 16.  Therefore the total is 16. Rotation 17 is not a rotation in and of itself but a place holder for weeks that can not be filled by the model.

We have 54 weeks for the whole period of the roster.  Let each week be $k$.

|j|Rotation Label|Duration|Maximum Interns per week|
|:-:|:-:|:-:|:-:|
|1|CPD-G|8|2|
|2|CPD-V|4|1|
|3|AP|4|1|
|4|MIC|4|1|
|5|MCH|2|1|
|6|CPCa|3|1|
|7|CPM|3|no limit|
|8|CPK|2|no limit|
|9|IP|4|2|
|10|DISP|3|no limit|
|11|CPC|5|no limit|
|12|QUM|1|1|
|13|H|1|1|
|14|A/L_1|1|11|
|15|A/L_2.1|1|6|
|16|A/L_2.2|1|5|
|17|Blank|-|-|




# v1.0

In v1.0 we were very inexperienced mathematical modellers, many of our decisions were based on a surface level understanding we retained from our introductory unit on mathematial modelling. We first approached the problem using strings as our parameters to be read from a seperate data file, our objective function was insufficent, and the format of our parameters was very limiting to our ability to complete the substasks that had been set out for us.

These are the dodgy parameters we set up for the model initially

{string}	Intern = ...;
{string}	Rotation = ...;
{string}	Week = ...;

$x_{jk}^i$ is our boolean decision variable which simply means it's either a 1 or a 0. $x_{11}^1$ = 1 would mean that Intern 1 is doing rotation 1 in week 1.

dvar boolean $x_{jk}^i$
where $i$ is an intern, rostered in area $j$, on week $k$

Our first constraint was to make sure that an Intern was not assigned multiple rotations in the same week.

$$ {\large\forall} \text{i in Intern, k in Week} \quad \sum_j^{Rotation} \; x^i_{j, k} \leq 1 $$

The lines below represent a completion constraint for the rotation CPD-G. It reads; for all interns, the sum over weeks for x[i]["CPD-G][k] must be greater than or equal to 8, essentially each intern must do 8 weeks of CPD-G throughout the year.

$$ \forall \text{i in Intern} \quad \sum_k^{Week} \; x_{"CPD-G",k}^i \geq 8 $$
  
This contstraint is what we call the Intern Rotation Capacity Constraint, it says that the for all weeks the sum of interns in CPD-G must be less than or equal to 2.  
  
$$ \forall \text{k in Week} \quad \sum_i^{Intern} \; x^i_{CPD-G, k} \leq 2 $$
  
This was our first attempt at a constraint for rotation duration, the logic behind is was that for all interns the sum over rotations and weeks for Duration[j] (the length of the rotation) multiplied by x[i][j][k] (whether a specific rotation is being done) must be greater than or equal to the sum of rotation in Duration[j]. This did not work how we expected and stumped us for weeks  
 
$$ \forall \text{i in Intern} \quad \sum_j^{Rotation} \sum_k^{Week} \; Duration_j * x^i_{j, k} \geq \sum_j^{Rotation} Duration_j $$

We were able to populate the weeks with the right number of rotations for the interns but when it came to having the rotations done in consecutive order the very nature of strings prevented us from moving any further in that direction

# v2.0

In v2.0 we removed our attempt at Rotation Duration constraints and introduced a Rotation Completition Constraint, the rotation completition constraint is a lot like the Intern Physical Constraint except it states that everyone must always be doing a job

$$ \forall \text{i in Intern, j in Rotation} \quad \sum_k^{Week} \; x^i_{j, k} \geq 1 $$

# v3.0

In v3.0 we ditched the string parameters and subsequently the data file the model was linked to and replaced them with a range of integers, using a range of integers instead of a string (list) of numbers allowed us to iterate through the range when creating Intern Rotation Duration Constraints although our initial formulation of them using Big M notation still did not produce the desired outcome and we were left with a model that would not run with 99+ errors. Big M notation allows us to use the logical operator IFTHEN.

We also created a new objective function which was to maximise the sum of all jobs being completed in all weeks by all interns.

$$ \text{maximise} \sum_i \; \sum_j \; \sum_k x_{jk}^i $$

The constraint below iterates through alpha when x is on and places all shifts of the same rotation after the first shift of the rotation, if rotation 1 is completed in one week then all 7 other rotation 1 shifts will be completed consecutively afterwards.

$$\sum_{\alpha = 0}^7 y_{1,k + \alpha}^i = 8 \; \text{  if  }\; x_{1,k}^i = 1$$



# v3.1

v3.1 we realised our Intern Rotation Duration Constraints needed a different y variable for each Big M constraints but also thought that we needed to use a different variable than x, our substitute x value was called w and w was exactly equal to x, this new x variable was not in fact needed.

$$\sum_{\alpha = 0}^7 y1_{1,k + \alpha}^i = 8 \; \text{  if  }\; x_{1,k}^i = 1$$

# v3.2

v3.2 we made the y values dependant on weeks $y_{[Week]}$ and increased Big M from 100 to 1000

$$8 - \sum_{\alpha = 0}^7 y1_{1,k + \alpha}^i \leq M\cdot (1-y1_{k})$$
$$\forall (i \; in\;Intern, k\;in\;1..47) \quad x_{j, k}^i \leq M\cdot y1_{k}$$

# v3.3

In v3.3 we stripped back the model to it's skeleton to test it's funcionality

# v4.0

In v4.0 added a dependance on Interns to the y variable, $y1_{Intern, Week}$ and a twist on the Intern Rotation Duration Constraints.

$$ \sum_k^{47} \; y1_{1, k} = 1$$

# v4.1

In v4.1 we optimised our rotation duration constraints.

# v4.2

In v4.2 we attempted to add orientation constraints which would only allow certain rotations (5, 9, 10) to be completed in the first 4 weeks of each intern group as 5 interns will start week 1 and go until week 50 and the other 6 interns will start week 4 and go until week 54

# v4.3

In v4.3 we introduced the first lot of annual leave constraints which required the addition of a new decision variable $L1_k$, the first leave constraint states that if one intern is to be on leave in a certain week then all other interns must also be on leave during that week.

sum(k in 9..50)L1[k] ==1;
forall(k in 9..50)
  11 - (sum(i in Intern)x[i][14][k]) <= M*(1-L1[k]);
  


# v4.3.1

In v4.3.1 we attempted to add the second week of leave, the second week of leave is to be split into two groups so not all interns will be taking their second week of leave together.

# v5.0

In v5.0 

## Decision Variables

$x_{jk}^i$
where $i$ is an intern, rostered in area $j$, on week $k$

Let $x_{jk}^i = 1$ if person $i$ is doing rotation $j$ for week $k$.

$x$ is valid when a person is beginning a rotation

$w_{j, k + \alpha}^i$
where $i$ is an intern, rostered in area $j$, on week $k$ and $\alpha$ is the duration of a rotation.

## Objective Function

$$ \text{maximise} \sum_i \; \sum_j \; \sum_k x_{jk}^i $$

## Constraints

### _Intern Physical Person Constraint_

That one person can only be in one place at once:

$$\sum_j \; x_{jk}^i \leq 1 \quad 
\forall i \; \forall k $$

### _Intern Rotation Completion Constraint_



$$\sum_k x_{1,k}^i \geq 8 \quad  \forall i $$

$$\sum_k x_{2,k}^i \geq 4 \quad  \forall i $$

$$\sum_k x_{3,k}^i = 4 \quad  \forall i $$

$$\sum_k x_{4,k}^i \geq 4 \quad  \forall i $$

$$\sum_k x_{5,k}^i \geq 2 \quad  \forall i $$

$$\sum_k x_{6,k}^i \geq 3 \quad  \forall i $$

$$\sum_k x_{7,k}^i \geq 3 \quad  \forall i $$

$$\sum_k x_{8,k}^i \geq 2 \quad  \forall i $$

$$\sum_k x_{9,k}^i \geq 4 \quad  \forall i $$

$$\sum_k x_{10,k}^i \geq 3 \quad  \forall i $$

$$\sum_k x_{11,k}^i \geq 5 \quad  \forall i $$

$$\sum_k x_{12,k}^i = 1 \quad  \forall i $$

$$\sum_k x_{13,k}^i =1 \quad  \forall i $$

$$\sum_k x_{14,k}^i =1 \quad  \forall i $$

$$\sum_k x_{15,k}^i =1 \quad  \forall i $$

$$\sum_k x_{16,k}^i =1 \quad  \forall i $$


### _Intern Rotation Capacity Constraint_
 
 The limit of how many interns can work in an area at once

$$\sum_{i}  x_{1,k}^i \leq 2  \quad  \; \forall k$$
$$\sum_{i}  x_{2,k}^i \leq 1  \quad  \; \forall k$$
$$\sum_{i}  x_{3,k}^i \leq 1  \quad  \; \forall k$$
$$\sum_{i}  x_{4,k}^i \leq 1  \quad  \; \forall k$$
$$\sum_{i}  x_{5,k}^i \leq 1  \quad  \; \forall k$$
$$\sum_{i}  x_{6,k}^i \leq 1  \quad  \; \forall k$$
$$\sum_{i}  x_{7,k}^i \geq 0  \quad  \; \forall k$$
$$\sum_{i}  x_{8,k}^i \geq 0  \quad  \; \forall k$$
$$\sum_{i}  x_{9,k}^i \leq 2  \quad  \; \forall k$$
$$\sum_{i}  x_{10,k}^i \geq 0  \quad  \; \forall k$$
$$\sum_{i}  x_{11,k}^i \geq 0  \quad  \; \forall k$$
$$\sum_{i}  x_{12,k}^i \leq 1  \quad  \; \forall k$$
$$\sum_{i}  x_{13,k}^i \leq 1  \quad  \; \forall k$$

### _Intern Rotation Duration Constraint_

This is the major point at which our current model falls apart.  We require interns to spend a period of time in each rotation in **consecutive** blocks.  Mathematically, the below system works, however it becomes flawed when trying to use it in CPLEX. This was an issue for two reasons:
1. The inability to input decision variables into "if" statements
    - This is something we have attemted to rectify with 'Big M' If/Then constraints.  This is currently still a work in progress, however we believe we have some issue with dependency of the variables - it would seem that the variables being dependent undermines the ability of If/Then to work correctly.
2. Our initial use of the $k$ values as strings. To combat the 2. issue, we attempted to change $k$ to an integer value, however this resulted in its inability to be used in the same fashion in $x_{jk}^i$. 
    - This has been resolved.
    


We are currently re-evaluating our options.  One that was explored was that we might just ascribe weights to each Rotation (this being the duration), and use a binary variable instead of $x_{ij}$.  However this limits us in the particular regard that we would not be able to specify specific weeks for availability - an aspect of flexibility integral as we refine the model.  There are to be a number of final constraints still to be added which depend on the model having this quality.

$$\sum_{\alpha = 0}^7 y_{1,k + \alpha}^i = 8 \; \text{  if  }\; x_{1,k}^i = 1$$

$$\sum_{\alpha = 0}^3 y_{2,k + \alpha}^i = 4 \; \text{  if  }\; x_{2,k}^i = 1$$

$$\sum_{\alpha = 0}^3 y_{3,k + \alpha}^i = 4 \; \text{  if  }\; x_{3,k}^i = 1$$

$$\sum_{\alpha = 0}^3 y_{4,k + \alpha}^i = 4 \; \text{  if  }\; x_{4,k}^i = 1$$

$$\sum_{\alpha = 0}^1 y_{5,k + \alpha}^i = 2 \; \text{  if  }\; x_{5,k}^i = 1$$

$$\sum_{\alpha = 0}^2 y_{6,k + \alpha}^i = 3 \; \text{  if  }\; x_{6,k}^i = 1$$

$$\sum_{\alpha = 0}^2 y_{7,k + \alpha}^i = 3 \; \text{  if  }\; x_{7,k}^i = 1$$

$$\sum_{\alpha = 0}^1 y_{8,k + \alpha}^i = 2 \; \text{  if  }\; x_{8,k}^i = 1$$

$$\sum_{\alpha = 0}^3 y_{9,k + \alpha}^i = 4 \; \text{  if  }\; x_{9,k}^i = 1$$

$$\sum_{\alpha = 0}^2 y_{10,k + \alpha}^i = 3 \; \text{  if  }\; x_{10,k}^i = 1$$

$$\sum_{\alpha = 0}^4 y_{11,k + \alpha}^i = 5 \; \text{  if  }\; x_{11,k}^i = 1$$

$$ y_{12,k}^i = 1 \; \text{  if  }\; x_{12,k}^i = 1$$

$$ y_{13,k}^i = 1 \; \text{  if  }\; x_{13,k}^i = 1$$

$$ y_{14,k}^i = 1 \; \text{  if  }\; x_{14,k}^i = 1$$

$$ y_{15,k}^i = 1 \; \text{  if  }\; x_{15,k}^i = 1$$

$$ y_{16,k}^i = 1 \; \text{  if  }\; x_{16,k}^i = 1$$

It should be noted that were the above constraints valid in the model, additional decision variables would be needed.

### Below are some experimentations of duration expressions

Expanding one of these...

$$ 8 - \sum_{\alpha = 0}^7 w_{1,k + \alpha}^i \leq M\cdot y \quad \forall_i \; \forall_{k=1}^{47}$$
$$ x_{1,k}^i \leq M\cdot (1-y) \quad \forall_i \; \forall_{k=1}^{47}$$

$$8 - \bigg( w_{1,1}^1 + w_{1,2}^1 + \cdots + w_{1,8}^1 \bigg) \leq M \cdot y$$
$$\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots $$
$$8 - \bigg( w_{1,47}^1 + w_{1,48}^1 + \cdots + w_{1,54}^1 \bigg) \leq M \cdot y$$
$$8 - \bigg( w_{1,1}^2 + w_{1,2}^2 + \cdots + w_{1,8}^2 \bigg) \leq M \cdot y$$
$$\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots $$
$$8 - \bigg( w_{1,47}^2 + w_{1,48}^2 + \cdots + w_{1,54}^2 \bigg) \leq M \cdot y$$
$$\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots $$
$$\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots $$
$$8 - \bigg( w_{1,47}^{11} + w_{1,48}^{11} + \cdots + w_{1,54}^{11} \bigg) \leq M \cdot y$$


$$ x_{1,1}^1 \leq M\cdot (1-y)$$
$$\vdots  \quad \quad \vdots \quad  \quad \vdots $$
$$ x_{1,47}^1 \leq M\cdot (1-y)$$
$$ x_{1,1}^2 \leq M\cdot (1-y)$$
$$\vdots  \quad \quad \vdots \quad  \quad \vdots $$
$$\vdots  \quad \quad \vdots \quad  \quad \vdots $$
$$ x_{1,47}^{11} \leq M\cdot (1-y)$$


There appears to be an issue here with having dependent aspects of the variables in the 'if' and 'then' statements - specifically.
In consequence we need an alternate formulation of the duration constraints.

In IBM CPLEX we currently have a constraint like this:

forall(i in Intern)
  sum(k in Week)x[i][1][k] >= 8;
  
However, this does not stipulate consecutive values.

$$ 8 - \sum_{\alpha = 0}^7 w_{1,k + \alpha}^i \leq M\cdot y_k \quad \forall_i \; \forall_{k=1}^{47}$$
$$ x_{1,k}^i \leq M\cdot (1-y_k) \quad \forall_i \; \forall_{k=1}^{47}$$

_for intern $1$, rotation $1$, week $1$_

$$ 8- \bigg( w_{1,1}^1 + w_{1,2}^1 + w_{1,3}^1 + w_{1,4}^1 + w_{1,5}^1 + w_{1,6}^1 + w_{1,7}^1 + w_{1,8}^1 \bigg) \leq 8 \cdot (1-y_k)$$
$$x_{1,1}^{1} \leq 1 \cdot y_k$$

when $y_1 = 0$

$$ 8- \bigg( w_{1,1}^1 + w_{1,2}^1 + w_{1,3}^1 + w_{1,4}^1 + w_{1,5}^1 + w_{1,6}^1 + w_{1,7}^1 + w_{1,8}^1 \bigg) \leq 8$$
$$x_{1,1}^{1} \leq 0$$

when $y_1 = 1$

$$ 8- \bigg( w_{1,1}^1 + w_{1,2}^1 + w_{1,3}^1 + w_{1,4}^1 + w_{1,5}^1 + w_{1,6}^1 + w_{1,7}^1 + w_{1,8}^1 \bigg) \leq 0$$
$$x_{1,1}^{1} \leq 1$$

We need an additional constraint (for each system?) to limit the y.

$${\Large \forall}_{i=0}^{11} \; \sum_{k=1}^{47} \; \sum_{\alpha = 0}^7 \; C_{1}^{i} \cdot  y_{1,(k + \alpha)}^i = 8   $$

$$C_{1}^{1} \cdot  y_{1,(k + \alpha)}^1 = 8$$

$$ 8 - \sum_{\alpha = 0}^7 w_{1,k + \alpha}^i \leq M\cdot y \quad \forall_i \; \forall_{k=1}^{47}$$
$$ x_{1,k}^i \leq M\cdot (1-y) \quad \forall_i \; \forall_{k=1}^{47}$$

_Intern Leave Constraint_

These contraints are currently flawed with our current model - particular as one is unable to use decision variables in 'if' statements and conditions


$$\sum_i x_{14,k}^i = 11z_k \quad \text{if} \quad \sum_k z_k =1$$

$$\sum_i x_{15,k}^i = 6z_k \quad \text{if} \quad \sum_k z_k =1$$

$$\sum_i x_{16,k}^i = 5z_k \quad \text{if} \quad \sum_k z_k =1$$

Additionally, after analysis we have deducted that if these constraints were to be working properly, there would be no need for those currently causing issue under the _Intern Rotation Capacity Constraint_.  We believe that those can be manipulated with little trouble however.